### makes spacecraft positions for THUX with carrington rotation longitude

In [1]:
import numpy as np
import scipy.io
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from matplotlib.dates import  DateFormatter
import seaborn as sns
import astropy
import astropy.constants as const
from sunpy.time import parse_time
import time
import pickle
import sys
import os
import urllib
import json
import importlib
import pandas as pd
import copy
import openpyxl
import h5py
import numba
from numba import jit
import multiprocessing
import heliopy.data.spice as spicedata
import heliopy.spice as spice
import astropy

import astropy.units as u
from astropy.coordinates import SkyCoord
import sunpy.coordinates
from sunpy.coordinates import frames



from heliocats import plot as hp
importlib.reload(hp) #reload again while debugging

from heliocats import data as hd
importlib.reload(hd) #reload again while debugging

from heliocats import cats as hc
importlib.reload(hc) #reload again while debugging

from heliocats import stats as hs
importlib.reload(hs) #reload again while debugging

#where the in situ data files are located is read 
#from config.py 
import config
importlib.reload(config)
from config import data_path
from config import data_path_ML

In [2]:
############################################ SETTINGS


#Coordinate System
#frame='HCI'
#frame='HEEQ'
frame='HEEQ'

print(frame)

#Time resolution
res_hours=1

print(res_hours)

################################## FUNCTIONS #############################################

@jit(nopython=True)
def sphere2cart(r, phi, theta):
    x = r*np.cos(theta)*np.cos(phi)
    y = r*np.cos(theta)*np.sin(phi)
    z = r*np.sin(theta)
    return (x, y, z) 

@jit(nopython=True)
def cart2sphere(x,y,z):
    r = np.sqrt(x**2+ y**2 + z**2)            # r
    theta = np.arctan2(z,np.sqrt(x**2+ y**2))     # theta
    phi = np.arctan2(y,x)                        # phi
    return (r, theta, phi)


plt.rcParams["figure.figsize"] = (20,10)



def convert_heeq_longitude_to_carrington(time, lon):

    heeq_lon = SkyCoord(np.degrees(lon)*u.deg, np.degrees(0)*u.deg, 1*u.AU, frame="heliographic_stonyhurst",  obstime=time)
    carr_lon=heeq_lon.transform_to(frames.HeliographicCarrington)
    
    return carr_lon.lon.rad



HEEQ
1


In [3]:
#https://docs.sunpy.org/en/latest/code_ref/coordinates/index.html#supported-coordinate-systems


start=time.time()
##########################################  PSP

starttime =datetime(2018, 8,13)
endtime = datetime(2025, 8, 31)
psp_time = []
while starttime < endtime:
    psp_time.append(starttime)
    starttime += timedelta(hours=res_hours)

spice.furnish(spicedata.get_kernel('psp_pred'))
psp=spice.Trajectory('SPP')
psp.generate_positions(psp_time,'Sun',frame)
print('PSP pos')

psp.change_units(astropy.units.AU)  
[psp_r, psp_lat, psp_lon]=cart2sphere(psp.x,psp.y,psp.z)


#to carrington longitude in radians
psp_lon_carr=convert_heeq_longitude_to_carrington(psp_time,psp_lon)

print('PSP conv')



#################################################### Solar Orbiter

starttime = datetime(2020, 3, 1)
endtime = datetime(2029, 12, 31)
solo_time = []
while starttime < endtime:
    solo_time.append(starttime)
    starttime += timedelta(hours=res_hours)

spice.furnish(spicedata.get_kernel('solo_2020'))
solo=spice.Trajectory('Solar Orbiter')
solo.generate_positions(solo_time, 'Sun',frame)
solo.change_units(astropy.units.AU)
[solo_r, solo_lat, solo_lon]=cart2sphere(solo.x,solo.y,solo.z)

#to carrington longitude in radians
solo_lon_carr=convert_heeq_longitude_to_carrington(solo_time,solo_lon)



print('Solo conv')



############################################## BepiColombo

starttime =datetime(2018, 10, 21)
endtime = datetime(2025, 11, 2)
bepi_time = []
while starttime < endtime:
    bepi_time.append(starttime)
    starttime += timedelta(hours=res_hours)

spice.furnish(spicedata.get_kernel('bepi_pred'))
bepi=spice.Trajectory('BEPICOLOMBO MPO') # or BEPICOLOMBO MMO
bepi.generate_positions(bepi_time,'Sun',frame)
bepi.change_units(astropy.units.AU)  
[bepi_r, bepi_lat, bepi_lon]=cart2sphere(bepi.x,bepi.y,bepi.z)
bepi_lon_carr=convert_heeq_longitude_to_carrington(bepi_time,bepi_lon)
print('Bepi done')


############# Earth for mercury, venus, STA
#https://docs.heliopy.org/en/stable/data/spice.html


#############stereo-B

starttime =datetime(2007, 1, 1)
endtime = datetime(2014, 9, 27)
stb_time = []
while starttime < endtime:
    stb_time.append(starttime)
    starttime += timedelta(hours=res_hours)
spice.furnish(spicedata.get_kernel('stereo_b'))
stb=spice.Trajectory('-235')  
stb.generate_positions(stb_time,'Sun',frame)  
stb.change_units(astropy.units.AU)  
[stb_r, stb_lat, stb_lon]=hd.cart2sphere(stb.x,stb.y,stb.z)
stb_lon_carr=convert_heeq_longitude_to_carrington(stb_time,stb_lon)
print('STEREO-B') 






planet_kernel=spicedata.get_kernel('planet_trajectories')
starttime =datetime(2007, 1, 1)
endtime = datetime(2029, 12, 31)
earth_time = []
while starttime < endtime:
    earth_time.append(starttime)
    starttime += timedelta(hours=res_hours)


earth=spice.Trajectory('399')  #399 for Earth, not barycenter (because of moon)
earth.generate_positions(earth_time,'Sun',frame)
earth.change_units(astropy.units.AU)  
[earth_r, earth_lat, earth_lon]=cart2sphere(earth.x,earth.y,earth.z)

earth_lon_carr=convert_heeq_longitude_to_carrington(earth_time,earth_lon)

print('Earth done')


################ mercury
mercury=spice.Trajectory('1')  #barycenter
mercury.generate_positions(earth_time,'Sun',frame)  
mercury.change_units(astropy.units.AU)  
[mercury_r, mercury_lat, mercury_lon]=hd.cart2sphere(mercury.x,mercury.y,mercury.z)
mercury_lon_carr=convert_heeq_longitude_to_carrington(earth_time,mercury_lon)
print('mercury') 

################# venus
venus=spice.Trajectory('2')  
venus.generate_positions(earth_time,'Sun',frame)  
venus.change_units(astropy.units.AU)  
[venus_r, venus_lat, venus_lon]=hd.cart2sphere(venus.x,venus.y,venus.z)
venus_lon_carr=convert_heeq_longitude_to_carrington(earth_time,venus_lon)
print('venus') 


############### Mars

mars=spice.Trajectory('4')  
mars.generate_positions(earth_time,'Sun',frame)  
mars.change_units(astropy.units.AU)  
[mars_r, mars_lat, mars_lon]=cart2sphere(mars.x,mars.y,mars.z)
mars_lon_carr=convert_heeq_longitude_to_carrington(earth_time,mars_lon)
print('mars done') 


#############stereo-A use 2 different kernels

starttime =datetime(2007, 1, 1)
endtime = datetime(2019, 12, 31)
sta1_time = []
while starttime < endtime:
    sta1_time.append(starttime)
    starttime += timedelta(hours=res_hours)
spice.furnish(spicedata.get_kernel('stereo_a'))
sta1=spice.Trajectory('-234')  
sta1.generate_positions(sta1_time,'Sun',frame)  
sta1.change_units(astropy.units.AU)  
[sta1_r, sta1_lat, sta1_lon]=hd.cart2sphere(sta1.x,sta1.y,sta1.z)
sta1_lon_carr=convert_heeq_longitude_to_carrington(sta1_time,sta1_lon)

starttime =datetime(2020, 1, 1)
endtime = datetime(2029, 12, 31)
sta2_time = []
while starttime < endtime:
    sta2_time.append(starttime)
    starttime += timedelta(hours=res_hours)

spice.furnish(spicedata.get_kernel('stereo_a_pred'))
sta2=spice.Trajectory('-234')  
sta2.generate_positions(sta2_time,'Sun',frame)  
sta2.change_units(astropy.units.AU)  
[sta2_r, sta2_lat, sta2_lon]=hd.cart2sphere(sta2.x,sta2.y,sta2.z)
sta2_lon_carr=convert_heeq_longitude_to_carrington(sta2_time,sta2_lon)


#add both
sta_time=np.hstack((sta1_time,sta2_time))
sta_r=np.hstack((sta1_r,sta2_r))
sta_lon=np.hstack((sta1_lon,sta2_lon))
sta_lat=np.hstack((sta1_lat,sta2_lat))
sta_lon_carr=np.hstack((sta1_lon_carr,sta2_lon_carr))



print('STEREO-A') 




end=time.time()
print( 'generate position took time in seconds:', round((end-start),1) )



psp=np.rec.array([np.array(psp_time),psp_r,psp_lon,psp_lat, psp_lon_carr],dtype=[('time','object'),('r','f8'),('lon','f8'),('lat','f8'),('lon_carr','f8')])
bepi=np.rec.array([np.array(bepi_time),bepi_r,bepi_lon,bepi_lat,bepi_lon_carr],dtype=[('time','object'),('r','f8'),('lon','f8'),('lat','f8'),('lon_carr','f8')])
solo=np.rec.array([np.array(solo_time),solo_r,solo_lon,solo_lat,solo_lon_carr],dtype=[('time','object'),('r','f8'),('lon','f8'),('lat','f8'),('lon_carr','f8')])

earth=np.rec.array([np.array(earth_time),earth_r,earth_lon,earth_lat, earth_lon_carr],dtype=[('time','object'),('r','f8'),('lon','f8'),('lat','f8'),('lon_carr','f8')])
mars=np.rec.array([np.array(earth_time),mars_r,mars_lon,mars_lat, mars_lon_carr],dtype=[('time','object'),('r','f8'),('lon','f8'),('lat','f8'),('lon_carr','f8')])
mercury=np.rec.array([np.array(earth_time),mercury_r,mercury_lon,mercury_lat,mercury_lon_carr],dtype=[('time','object'),('r','f8'),('lon','f8'),('lat','f8'),('lon_carr','f8')])
venus=np.rec.array([np.array(earth_time),venus_r,venus_lon,venus_lat, venus_lon_carr],dtype=[('time','object'),('r','f8'),('lon','f8'),('lat','f8'),('lon_carr','f8')])

sta=np.rec.array([np.array(sta_time),sta_r,sta_lon,sta_lat, sta_lon_carr],dtype=[('time','object'),('r','f8'),('lon','f8'),('lat','f8'),('lon_carr','f8')])
stb=np.rec.array([np.array(stb_time),stb_r,stb_lon,stb_lat, stb_lon_carr],dtype=[('time','object'),('r','f8'),('lon','f8'),('lat','f8'),('lon_carr','f8')])



pickle.dump([psp, bepi, solo, earth, mars, mercury,venus,sta,stb], open( 'results/positions_'+frame+'_1hr_carrington.p', "wb" ) )


psp.time= parse_time(psp.time).isot   
bepi.time= parse_time(bepi.time).isot   
solo.time= parse_time(solo.time).isot   
earth.time= parse_time(earth.time).isot
mars.time= parse_time(mars.time).isot
mercury.time= parse_time(mercury.time).isot
venus.time= parse_time(venus.time).isot
sta.time= parse_time(sta.time).isot
stb.time= parse_time(stb.time).isot




np.savetxt('results/positions_ascii/psp_'+frame+'_1hr_carrington.txt',psp,header='time, r (AU), lon (rad), lat (rad), lon_carr (rad), frame:'+frame,fmt='%16s %.18e %.18e %.18e %.18e ')
np.savetxt('results/positions_ascii/bepi_'+frame+'_1hr_carrington.txt',bepi,header='time, r (AU), lon (rad), lat (rad), lon_carr (rad), frame:'+frame,fmt='%16s %.18e %.18e %.18e %.18e ')
np.savetxt('results/positions_ascii/solo_'+frame+'_1hr_carrington.txt',solo,header='time, r (AU), lon (rad), lat (rad), lon_carr (rad), frame:'+frame,fmt='%16s %.18e %.18e %.18e %.18e ')
np.savetxt('results/positions_ascii/earth_'+frame+'_1hr_carrington.txt',earth,header='time, r (AU), lon (rad), lat (rad), lon_carr (rad), frame:'+frame,fmt='%16s %.18e %.18e %.18e %.18e ')
np.savetxt('results/positions_ascii/mars_'+frame+'_1hr_carrington.txt',mars,header='time, r (AU), lon (rad), lat (rad), lon_carr (rad), frame:'+frame,fmt='%16s %.18e %.18e %.18e %.18e')
np.savetxt('results/positions_ascii/mercury_'+frame+'_1hr_carrington.txt',mercury,header='time, r (AU), lon (rad), lat (rad), lon_carr (rad), frame:'+frame,fmt='%16s %.18e %.18e %.18e %.18e ')
np.savetxt('results/positions_ascii/venus_'+frame+'_1hr_carrington.txt',venus,header='time, r (AU), lon (rad), lat (rad), lon_carr (rad), frame:'+frame,fmt='%16s %.18e %.18e %.18e %.18e ')
np.savetxt('results/positions_ascii/stereoa_'+frame+'_1hr_carrington.txt',sta,header='time, r (AU), lon (rad), lat (rad), lon_carr (rad), frame:'+frame,fmt='%16s %.18e %.18e %.18e %.18e ')
np.savetxt('results/positions_ascii/stereob_'+frame+'_1hr_carrington.txt',stb,header='time, r (AU), lon (rad), lat (rad), lon_carr (rad), frame:'+frame,fmt='%16s %.18e %.18e %.18e %.18e ')





plt.figure(1)
plt.title('PSP')
plt.plot(psp_time,np.degrees(psp_lon))
plt.plot(psp_time,np.degrees(psp_lon_carr))

plt.figure(2)
plt.title('SolO')
plt.plot(solo_time,np.degrees(solo_lon))
plt.plot(solo_time,np.degrees(solo_lon_carr))


plt.figure(3)
plt.title('Bepi')
plt.plot(bepi_time,np.degrees(bepi_lon))
plt.plot(bepi_time,np.degrees(bepi_lon_carr))


plt.figure(4)
plt.title('Earth')
plt.plot(earth_time,np.degrees(earth_lon))
plt.plot(earth_time,np.degrees(earth_lon_carr))

plt.figure(5)
plt.title('Mars')
plt.plot(earth_time,np.degrees(mars_lon))
plt.plot(earth_time,np.degrees(mars_lon_carr))

PSP pos


PSP conv


Solo conv


Bepi done
STEREO-B


Earth done
mercury
venus
mars done


SpiceyError: 
================================================================================

Toolkit version: N0066

SPICE(SPKINSUFFDATA) --

Insufficient ephemeris data has been loaded to compute the state of -234 (STEREO AHEAD) relative to 10 (SUN) at the ephemeris epoch 2020 MAY 04 18:01:09.185.

spkezr_c --> SPKEZR --> SPKEZ --> SPKGEO

================================================================================